# OpenAI Assistant



In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
# make sure you have a .env file prepared on your local machine with the OPENAI API Key, Amadeus API Client Id and Client Secret
#upload your .env file
from google.colab import files

uploaded = files.upload()

Saving key.env to key.env


In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('key.env')

# Now you can retrieve your variables
OPENAI = os.getenv('OPENAI')

import openai
import json

openai.api_key = OPENAI

In [ ]:
!pip install amadeus

  Preparing metadata (setup.py) ... done
  Created wheel for amadeus: filename=amadeus-9.0.0-py2.py3-none-any.whl size=75047 sha256=78d776f3a519c6a354658388e5717bd9690f22ee7c7a35426d87f7566dcff97f
  Stored in directory: /root/.cache/pip/wheels/b0/50/ea/3417d93eee6760a945d7711333d8d42b9f482e84600ef7f711
Successfully built amadeus


In [ ]:
from amadeus import Client, ResponseError

In [ ]:
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [ ]:
amadeus = Client(
    client_id= CLIENT_ID,
    client_secret= CLIENT_SECRET
)

In [ ]:
def city_code_search(city_name):
    try:
        response = amadeus.reference_data.locations.get(keyword=city_name, subType='CITY')
        return response.data[0]['address']['cityCode']
    except ResponseError as error:
        return(error)

In [ ]:
def hotel_city_search(cityCode):
    try:
        response = amadeus.reference_data.locations.hotels.by_city.get(cityCode=cityCode)
        hotel_names = [item['hotelId'] for item in response.data[:50]]
        return hotel_names

    except ResponseError as error:
        return(error)


In [ ]:
from datetime import date, timedelta

def hotel_offers_search(hotelIds, checkInDate = date.today() ,
                        checkOutDate = date.today() + timedelta(days=1)
                       ):
    try:
        response = amadeus.shopping.hotel_offers_search.get(
            hotelIds= hotelIds,
            checkInDate = checkInDate,
            checkOutDate = checkOutDate
        )
        return(response.data)
    except ResponseError as error:
        return(error)


In [ ]:
def get_hotel_offers(location, check_in_date = date.today() ,
                        check_out_date = date.today() + timedelta(days=1)
                       ):
    hotel_offers = hotel_offers_search( hotel_city_search(city_code_search(location)), check_in_date, check_out_date)
    return json.dumps(hotel_offers)

In [ ]:
# define a tool
tools = [
        {
        "type":"function",
        "function" : {
          "name": "get_hotel_offers",
          "description": "Get the hotel offers in a given location",
          "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "city/state, e.g. Portland/OR",
                },
                "check_in_date": {
                    "type" : "string",
                    "description": "Check-in date of the stay (hotel local date). Format YYYY-MM-DD.\
                      The lowest accepted value is the present date (no dates in the past).\
                      If not present, the default value will be today's date in the GMT time zone. \
                      Example : 2023-11-22"
                },
                "check_out_date": {
                    "type" : "string",
                    "description": "Check-out date of the stay (hotel local date). Format YYYY-MM-DD.\
                      The lowest accepted value is check_in_date+1. If not present, it will default to check_in_date +1. Example : 2023-11-23 "
                },
            },
            "required": ["location"],
        },
      }
    }
]

In [ ]:
from openai import OpenAI
client = openai

In [ ]:
assistant = client.beta.assistants.create(
  instructions="You are a travel agent. Use the provided functions to answer questions. Please do not share the booking link",
  model="gpt-3.5-turbo-0613",
  tools= tools
)

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
        "role": "user",
        "content": "Hello,  I want to go to Las Vegas, arrive 11/22, departure 11/25, please find hotels for me.  Thank you."
    }
]
)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

 # Poll for the run to complete and retrieve the assistant's messages

In [ ]:
import time
while run.status != 'completed':
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
            if run.status == "requires_action":
              required_action = run.required_action

              # Check if the required action is 'submit_tool_outputs'
              if required_action.type == "submit_tool_outputs":
                tool_outputs = []
                for tool_call in required_action.submit_tool_outputs.tool_calls:
                # Check if the tool call is a function
                  if tool_call.type == "function":
                    function_name = tool_call.function.name
                    arguments_json = tool_call.function.arguments

                    # Parse the JSON arguments
                    arguments = json.loads(arguments_json)

                    # Assuming you have a function mapping
                    function_mapping = {
                      "get_hotel_offers": get_hotel_offers,  # get_hotel_offers is a function defined in your code
                      # ... other function mappings
                    }

                    # Check if the function exists and call it
                    if function_name in function_mapping:
                      print('Calling function: ', function_name,arguments_json )
                      response = function_mapping[function_name](**arguments)
                      print('Function response: ', response)
                      tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": response,
                      })
                    # submit the tool outputs to the thread and run
                    print('submit the tool outputs to the thread and run')
                    run = client.beta.threads.runs.submit_tool_outputs(
                      thread_id=thread.id,
                      run_id=run.id,
                      tool_outputs= tool_outputs
                    )


Calling function:  get_hotel_offers {
  "location": "Las Vegas",
  "check_in_date": "2023-11-22",
  "check_out_date": "2023-11-25"
}
Function response:  [{"type": "hotel-offers", "hotel": {"type": "hotel", "hotelId": "BWLAS058", "chainCode": "BW", "dupeId": "700070959", "name": "Best Western McCarran Inn", "cityCode": "LAS", "latitude": 36.1, "longitude": -115.14932}, "available": true, "offers": [{"id": "L4Y9ZEK2FJ", "checkInDate": "2023-11-22", "checkOutDate": "2023-11-25", "rateCode": "RAC", "rateFamilyEstimated": {"code": "BAR", "type": "P"}, "commission": {"percentage": "10"}, "boardType": "BREAKFAST", "room": {"type": "A1K", "typeEstimated": {"category": "STANDARD_ROOM", "beds": 1, "bedType": "KING"}, "description": {"text": "FLEXIBLE RATE*BEST LEAST RESTRICTIVE RATE\n1 KING BED,MOBACC,COMAST,ROLLSH,NSMK,\nFULL BREAKFAST", "lang": "EN"}}, "guests": {"adults": 1}, "price": {"currency": "USD", "base": "297.00", "total": "352.50", "taxes": [{"code": "TOTAL_TAX", "amount": "55.50", "

In [ ]:
reply = client.beta.threads.messages.list(
  thread_id=thread.id
)
messages = reply.data

assistant_reply = ""
for message in messages:
    if message.role == "assistant":  # Check if the message is from the assistant
        for content in message.content:
            if content.type == "text":
                assistant_reply = content.text.value
                break  # Assuming only one text content per assistant's message
        if assistant_reply:
            break  # Break the loop if the assistant's reply is found

print("Assistant's Reply:", assistant_reply)


Assistant's Reply: I found some hotel options for you in Las Vegas from 11/22 to 11/25:

1. Best Western McCarran Inn
   - Room Type: Standard Room with 1 King Bed
   - Description: Flexible Rate, Best Least Restrictive Rate
   - Price: $352.50 (including taxes)
   - Cancellation Policy: Free cancellation until 11/21 at 4:00 PM
   - [More details and booking](https://test.api.amadeus.com/v3/shopping/hotel-offers/L4Y9ZEK2FJ)

2. Best Western Plus Las Vegas West
   - Room Type: Standard Room with 1 King Bed
   - Description: Multi Night Stay Promotion
   - Price: $366.91 (including taxes)
   - Cancellation Policy: Non-refundable rate
   - [More details and booking](https://test.api.amadeus.com/v3/shopping/hotel-offers/B139U5NDKJ)

3. Best Western Plus Casino Royale–Center Strip
   - Room Type: Standard Room with 1 King Bed
   - Description: Advance Purchase
   - Price: $261.86 (including taxes)
   - Cancellation Policy: Non-refundable rate
   - [More details and booking](https://test.api